In [1]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import uproot
import os
import glob
import awkward as ak
from scipy.stats import norm

In [2]:
setup_path ="/home/haue/repositories/Masters/setup/" 
data_path = "/home/haue/repositories/Masters/data/singleprotons/Al/inclbetacut/"
twop_path = "/home/haue/repositories/Masters/data/twoproton/Al/"
Al_files = glob.glob(os.path.join(data_path, "*mlio.root"))
twop_files = glob.glob(os.path.join(twop_path, "*mlio.root"))


frac = 0.9542011788469517 # (Na21/(p+Na21)) amount of energy the daughter nucleus gets of the total energy released in the decay
frac_err = 1.032071836027589e-10 # uncertainty of the fraction
psep = 5.50410 # proton separation energy of 22Mg
psep_err = 0.00016 # error of proton separation energy of 22Mg
scale = 3.326542500626603 # difference in solid angle coverage of two proton events up to 4pi
sac4 = 6.795917343263473 # solid angle coverage of id3

In [3]:
E2_id3 = np.array([])
for batch in uproot.iterate(Al_files, expressions=["E"],
                            cut="(id==3)"):
    E2_id3=np.append(E2_id3, ak.flatten(batch.E))
E2p_id3 = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"],
                            cut="(id==3)"):
    E2p_id3=np.append(E2p_id3, ak.flatten(batch.E))
    
E2_id2 = np.array([])
for batch in uproot.iterate(Al_files, expressions=["E"],
                            cut="(id==2)"):
    E2_id2=np.append(E2_id2, ak.flatten(batch.E))
E2p_id2 = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"],
                            cut="(id==2)"):
    E2p_id2=np.append(E2p_id2, ak.flatten(batch.E))




In [5]:
y1p_id3, _ = np.histogram(E2_id3, bins=np.arange(0,9000+10,10))
y2p_id3, _ = np.histogram(E2p_id3, bins=np.arange(0,9000+10,10))
y1p_id2, _ = np.histogram(E2_id2, bins=np.arange(0,1000+10,10)) #to find the missing counts from id3 spectrum
y2p_id2, _ = np.histogram(E2p_id2, bins=np.arange(0,1000+10,10)) # to find the missing counts from id3 spectrum

counts_1p_id3 = np.sum(y1p_id3); counts_1p_id3_err = np.sqrt(np.sum(y1p_id3))
counts_2p_id3 = np.sum(y2p_id3); counts_2p_id3_err = np.sqrt(np.sum(y2p_id3))
counts_1p_id2 = np.sum(y1p_id2); counts_1p_id2_err = np.sqrt(np.sum(y1p_id2))
counts_2p_id2 = np.sum(y2p_id2); counts_2p_id2_err = np.sqrt(np.sum(y2p_id2))
relative_diff = 11122/9083 # relative peaksizes of 1.245 peak of id3/id2 186, 290
#relative_diff_err = 0.05
relative_diff_err = np.sqrt((186 / 9083)**2 + (11122 * 290 / 9083**2)**2+0.05**2)
#print(relative_diff_err)
relative_solidangle = 100/sac4

tot_1p_counts = (counts_1p_id3-counts_2p_id3*scale + counts_1p_id2*relative_diff-counts_2p_id2*scale*relative_diff ) * relative_solidangle # total 1p counts in id3 where the region below 1MeV has been estimated from id2
tot_1p_counts_err = np.sqrt((counts_1p_id3_err)**2 + (counts_2p_id3_err * scale)**2 + (counts_1p_id2_err * relative_diff)**2 + (counts_2p_id2_err * scale * relative_diff)**2 + (counts_1p_id2 * relative_diff_err)**2 + (counts_2p_id2 * scale * relative_diff_err)**2) * relative_solidangle

tot_2p_counts = ((counts_2p_id3*scale+counts_2p_id2*scale*relative_diff) * relative_solidangle) /2 # divided by 2 because 1 2p event is two protons
tot_2p_counts_err = 0.5 * relative_solidangle * np.sqrt((counts_2p_id3_err * scale)**2 + (counts_2p_id2_err * scale * relative_diff)**2 + (counts_2p_id2 * scale * relative_diff_err)**2)


tot_g_counts = 534541# +/- 17814.62886138098 - weighted mean calculated from counts in 583 peak in G1 & G2
tot_g_counts_err = 17815

tot = tot_1p_counts+tot_2p_counts+tot_g_counts
tot_err = np.sqrt(tot_1p_counts_err**2+tot_2p_counts_err**2+tot_g_counts_err**2)
print(f"Total number of events recorded(not including alphas yet): {tot:.0f} +/- {tot_err:.2f}")
print(f"Branching ratio 1p: {tot_1p_counts/tot*100:.2f} +/- {100 * np.sqrt((tot_1p_counts_err / tot)**2 + (tot_1p_counts * tot_err / tot**2)**2):.2f}")
print(f"Branching ratio 2p: {tot_2p_counts/tot*100:.2f} +/- {100 * np.sqrt((tot_2p_counts_err / tot)**2 + (tot_2p_counts * tot_err / tot**2)**2):.2f}")
print(f"Branching ratio g: {tot_g_counts/tot*100:.2f} +/- {100 * np.sqrt((tot_g_counts_err / tot)**2 + (tot_g_counts * tot_err / tot**2)**2):.2f}")
print()
print("Total proton events", tot-tot_g_counts)

Total number of events recorded(not including alphas yet): 1150404 +/- 20426.44
Branching ratio 1p: 49.96 +/- 1.24
Branching ratio 2p: 3.57 +/- 0.11
Branching ratio g: 46.47 +/- 1.75

Total proton events 615862.5714665635


In [ ]:
ps3 = [1249,1167,1311,2486,2396,2563,5456,3880]
cs3 = [11122,304,591,3740,66,129,236,363]
ce3 = [186,32,50,115,7,24,25,39]
for c,p,e in zip(cs3,ps3,ce3):
    print(f"Peak E={p} keV -- Branching ratio: {c/(tot_1p_counts/relative_solidangle)*100:.2f} +/- {(c * relative_solidangle / tot_1p_counts * 100) * np.sqrt((e / c)**2 + (tot_1p_counts_err / tot_1p_counts)**2):.2f}")


Peak E=1249 keV -- Branching ratio: 28.47 +/- 0.69
Peak E=1167 keV -- Branching ratio: 0.78 +/- 0.08
Peak E=1311 keV -- Branching ratio: 1.51 +/- 0.13
Peak E=2486 keV -- Branching ratio: 9.57 +/- 0.34
Peak E=2396 keV -- Branching ratio: 0.17 +/- 0.02
Peak E=2563 keV -- Branching ratio: 0.33 +/- 0.06
Peak E=5456 keV -- Branching ratio: 0.60 +/- 0.06
Peak E=3880 keV -- Branching ratio: 0.93 +/- 0.10


In [57]:
def rel(totc,totc_e,c, c_e):
    br = (c/totc) *100
    br_e = br*np.sqrt((c_e/c)**2+(totc_e/totc)**2)
    return br,br_e

def weighted_mean(values, errors):
    values = np.asarray(values)
    errors = np.asarray(errors)
    weights = 1.0 / (errors**2)
    mean = np.sum(weights * values) / np.sum(weights)
    err_mean = np.sqrt(1.0 / np.sum(weights))
    return mean, err_mean

ps3 = [1249,1167,1311,2486,2396,2563,5456,3880,1670]
cs3 = [11122,304,591,3740,66,129,236,363,597]
ce3 = [186,32,50,115,7,24,25,39,47]

cs2 = [9083,115,179,197,353,1346,255,3243,1899,335]
ce2 = [290,21,10,32,43,53,27,102,77,26]
ps2 = [1249,1167,1311,5456,3880,996,775,693,452,1670]

cs1 = [8617,146,116,224,334,1184,252,2232,324]
ce1 = [277,17,6,28,40,69,29,59, 33]
ps1 = [1249,1167,1311,5456,3880,996,775,693,1670]

"""
for c,p,e in zip(cs1,ps1,ce1):
    print(p, rel(cs1[0], ce1[0], c,e))

    
print("="*60)
for c,p,e in zip(cs2,ps2,ce2):
    print(p, rel(cs2[0], ce2[0], c,e))


print("="*60)
for c,p,e in zip(cs3,ps3,ce3):
    print(p, rel(cs3[0], ce3[0], c,e))
print("="*60)
print("="*60)

"""
scale = 28.47 / 100 # scale to absolute
scale_err = 0.69 / 100
p1249 = weighted_mean([100,100,100],[4.515269548477348,4.546096748025383,2.3650757291979474])
p1167 = weighted_mean([1.2661015083122316,1.694325171173262,2.7333213450818197],[0.23470844383175457,0.20466466631903152,0.29132655934372054])
p1311 = weighted_mean([1.9707145216338213,1.3461761633979343,5.313792483366301], [0.12680723606879965,0.08198131298001363,0.45825845620197186])
p5456 = weighted_mean([2.1688869316305186,2.5995125913891144,2.1219205178924656], [0.3590475232387749,0.335511888737717,0.2275635936304276])
p3880 = weighted_mean([3.8863811516018933,3.8760589532319833,0.2638014745549357], [0.4894032231917866,0.4806300930302771,0.3548790425599869])
p996 = weighted_mean([14.818892436419684, 13.740280840199606], [0.7512238289459661,0.9144837344189546]) # id1,id2
p775 = weighted_mean([2.807442474953209, 2.924451665312754], [0.3104789666274419, 0.349427432002272]) # id1,id2
p693 = weighted_mean([35.70406253440493, 25.902286178484392], [1.6001769921321851, 1.078011278717349]) # id1,id2
p452 = weighted_mean(20.90718925465155, 1.0790004538299827) #id2
p2486 = weighted_mean(33.62704549541449, 1.177023183983781) #id3
p2396 =weighted_mean(0.5934184499190793, 0.06371593149419423) #id3
p2563 =weighted_mean(1.159863333932746, 0.2166585703278973) #id3
p1670 = weighted_mean([3.760009283973541,3.6882087416051963,5.367739615177126], [0.4015850013106685,0.3095239228095938,0.43201516846290305])
peaks = [p1249,p1167,p1311,p5456,p3880,p996,p775,p693,p452,p2486,p2396,p2563,p1670]
peakvals = [1249,1167,1311,5456,3880,996,775,693,452,2486,2396,2563,1673]
brs = []
for (mean, err),p in zip(peaks,peakvals):
    abs_mean = mean * scale
    abs_err = abs_mean * np.sqrt((err / mean)**2 + (scale_err / scale)**2)
    brs.append(abs_mean)
    print(f"E={p} - {abs_mean:.2f} +/- {abs_err:.2f}")
print(f"Total branching ratio accounted for: {np.sum(brs):.2f}")

E=1249 - 28.47 +/- 0.88
E=1167 - 0.51 +/- 0.04
E=1311 - 0.46 +/- 0.02
E=5456 - 0.64 +/- 0.05
E=3880 - 0.61 +/- 0.07
E=996 - 4.10 +/- 0.19
E=775 - 0.81 +/- 0.07
E=693 - 8.25 +/- 0.32
E=452 - 5.95 +/- 0.34
E=2486 - 9.57 +/- 0.41
E=2396 - 0.17 +/- 0.02
E=2563 - 0.33 +/- 0.06
E=1673 - 1.17 +/- 0.07
Total branching ratio accounted for: 61.04
